In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
user = 'root'
password = 'test'
host = 'localhost'
database = 'insightdb'

In [4]:
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}')

In [5]:
import mysql.connector

In [6]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="test",
  database="utdacademicdata"
)

In [7]:
cursor = mydb.cursor()

In [15]:
cursor.execute("SET FOREIGN_KEY_CHECKS = 0;")

In [77]:
#cursor.execute("DROP TABLE courses")
cursor.execute("DROP TABLE professors")
# cursor.execute("DROP TABLE grades_by_professor")
# cursor.execute("DROP TABLE professor_course_mapping")
#cursor.execute("DROP TABLE profs_remarks")

In [42]:
cursor.execute("create table courses (course_id VARCHAR(255), title VARCHAR(255), class_level VARCHAR(255), PRIMARY KEY(course_id));")

ProgrammingError: 1050 (42S01): Table 'courses' already exists

In [22]:
courses_df = pd.read_sql_table('courses', engine)

In [25]:
courses_df['course_id'] = courses_df['subject_prefix'] + courses_df['course_number']
courses_df.head()

,subject_prefix,course_number,title,class_level,course_id
0,ACCT,2301,Introductory Financial Accounting,Undergraduate,ACCT2301
1,ACCT,2302,Introductory Management Accounting,Undergraduate,ACCT2302
2,ACCT,3100,Professional Development,Undergraduate,ACCT3100
3,ACCT,3101,Professional Program in Accounting Lyceum,Undergraduate,ACCT3101
4,ACCT,3200,Introduction to Business Professional Developm...,Undergraduate,ACCT3200


In [26]:
courses_df.drop(['subject_prefix', 'course_number'], axis=1, inplace=True)

In [31]:
sql = "INSERT into courses(course_id, title, class_level) values(%s, %s, %s)"
for index, row in courses_df.iterrows():
    val = (row['course_id'], row["title"], row["class_level"])
    cursor.execute(sql, val)

In [120]:
cursor.execute("DROP TABLE professors")


In [121]:
cursor.execute("create table professors (id VARCHAR(255) NOT NULL, name VARCHAR(255) NOT NULL, email VARCHAR(255), phone_number VARCHAR(255), PRIMARY KEY(name));")

In [34]:
professors_df = pd.read_sql_table('professors', engine)

In [51]:
professors_df['profFirst'] = professors_df['profFirst'].astype('str')
professors_df['profLast'] = professors_df['profLast'].astype('str')

In [112]:
professors_df.drop(3318, inplace=True)

In [119]:
professors_df['name'] = professors_df['profFirst'] + professors_df['profLast']
professors_df.drop(['profFirst', 'profLast'], axis=1, inplace=True)

In [57]:
professors_df.drop_duplicates(subset=['profFirst', 'profLast'], inplace=True)

In [122]:
sql = "INSERT into professors(id, name, email, phone_number) values(%s, %s, %s, %s)"
for index, row in professors_df.iterrows():
    val = (row["professorId"], row["name"], row["email"], row['phone_number'])
    cursor.execute(sql, val)

In [130]:
cursor.execute("DROP TABLE grades")


In [131]:
cursor.execute("create table grades (prof_name VARCHAR(255) NOT NULL, course_id VARCHAR(255) NOT NULL, term VARCHAR(255) NOT NULL, Aplus int, A int, Aminus int, Bplus int, B int, Bminus int, Cplus int, C int, Cminus int, Dplus int, D int, Dminus int, F int, W int, PRIMARY KEY(prof_name, course_id, term), FOREIGN KEY (prof_name) References professors(name), FOREIGN KEY (course_id) references courses(course_id));")

In [124]:
grades_df = pd.read_sql_table('grades_by_professor', engine)

In [126]:
grades_df['prof_name'] = grades_df['profFirst'] + grades_df['profLast']
grades_df['course_id'] = grades_df['subject_prefix'] + grades_df['course_number']

In [127]:
grades_df = grades_df[grades_df['prof_name'].isin(professors_df['name'])]

In [136]:
sql = "INSERT into grades(prof_name, course_id, term, Aplus, A, Aminus, Bplus, B, Bminus, Cplus, C, Cminus, Dplus, D, Dminus, F, W) values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
for index, row in grades_df.iterrows():
    val = (row["prof_name"], row['course_id'], row["term"], row['A+'], row["A"], row["A-"], row["B+"], row["B"], row["B-"], row["C+"], row["C"], row["C-"], row["D+"], row["D"], row["D-"], row["F"], row["W"])
    cursor.execute(sql, val)

In [153]:
cursor.execute('DROP table professor_remarks')

In [154]:
cursor.execute("create table professor_remarks (prof_name VARCHAR(255) NOT NULL, avg_difficulty double, avg_rating double, department VARCHAR(255), total_rating int, r1 int, r2 int, r3 int, r4 int, r5 int, tags TEXT, would_take_again double, PRIMARY KEY(prof_name), FOREIGN KEY(prof_name) References professors(name));")

In [139]:
remarks_df = pd.read_sql_table('profs_remarks', engine)

In [140]:
remarks_df['prof_name'] = remarks_df['profFirst'] + remarks_df['profLast']

In [155]:
sql = "INSERT into professor_remarks(prof_name, avg_difficulty, avg_rating, department, total_rating, r1, r2, r3, r4, r5, tags, would_take_again) values(%s, %s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s)"
for index, row in remarks_df.iterrows():
    val = (row["prof_name"], row["avgDifficulty"], row["avgRating"], row['department'], row['totalRatings'], row['r1'], row['r2'], row['r3'], row['r4'], row['r5'], row['professorTags'], row['wouldTakeAgainPercent'])
    cursor.execute(sql, val)

In [152]:
remarks_df['prof_name'] = remarks_df['prof_name'].astype('str')
remarks_df.drop_duplicates(subset='prof_name', inplace=True) 

In [156]:
cursor.close()

True

In [157]:
mydb.commit()
mydb.close()